<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/GetCsvFromJsonOnline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Récupération du fichier JSON fourni par lemonde prenant la forme d'un dictionnaire contenant des informations sur différents sites de news.

https://www.lemonde.fr/webservice/decodex/updates

In [53]:
import urllib.request, json 
with urllib.request.urlopen("https://www.lemonde.fr/webservice/decodex/updates") as url:
    data = json.loads(url.read().decode())

Création de la liste ***dataset*** contenant pour tous les sites du fichier JSON :
*   Leur identifiant
*   Leur nom
*   Leur niveau de fiabilité



In [54]:
import pandas as pd

dataset = []

for i in data["sites"].keys():
  dataset.append([int(i),data["sites"][i][2],data["sites"][i][0]])

Récupération dans une liste ***df*** de tous les liens Twitter fournis dans le JSON

In [55]:
import re

dt = []
compteur = 0
for i in data["urls"].keys():
  if not(re.search('twitter', i)):
    dt.append(compteur)
  compteur = compteur+1

df = [(data["urls"][i], i) for i in data["urls"].keys()]

for index in sorted(dt, reverse=True):
    del df[index]

for site in dataset:
  if site[0] not in [item for t in df for item in t]:
    df.append((site[0], "NaN"))
df.sort(reverse=True)

Suppression dans ***dataset*** des lignes dont le site ne possède pas de lien twitter conforme.

Ajout dans ***dataset*** de la colonne userID contenant le nom d'utilisateur Twitter pour les sites restant.

In [56]:
import copy
save_dataset = copy.deepcopy(dataset)

i = 0
for i in range(len(save_dataset)):
  save_dataset[i] = save_dataset[i] + [df[i][1]]
  i += 1

save_dataset = [x for x in save_dataset if x[3] != "NaN"]

for line in save_dataset:
  if "twitter.com/" in line[3]:
    if "?lang=fr" in line[3]:
      size = len(line[3])
      line[3] = line[3][12:size - 8].strip('@')
    else:
      line[3] = line[3][12:].strip('@')

Sauvegarde sur le Drive de ***dataset*** dans un fichier csv

Le niveau de fiabilité des sites se lie de cette manière :


1.   Site parodique
2.   Site non fiable
3.   Site à la fiabilité douteuse
4.   Site fiable



In [57]:
from google.colab import drive
drive.mount('/content/drive')

res = pd.DataFrame(save_dataset, columns=['id','name','trust','userID'])

res.to_csv('tableauSites.csv')
!cp tableauSites.csv "/content/drive/My Drive/PFE"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
